In [ ]:
#using Revise

In [1]:
using Knet

In [2]:
#pkg"add Knet#master"

In [3]:
#using ImageView;

In [14]:
include("../helper.jl")
include("accuracy.jl")
include("loss.jl")
include("mpii.batcher.jl")
include("../models.jl")
include("../training.jl")
include("../plots.jl")
include("../modular.resnet.jl")

generate_headless_resnet50_from_weights (generic function with 1 method)

In [15]:
dtrn_2k, dval_2k, data_items = get_mpii_batches_and_data_items(32; should_shuffle=true)
dtrn_x1_2k, dtrn_y1_2k = first(dtrn_2k)
dval_x1_2k, dval_y1_2k = first(dval_2k)

1
8-element Data{Tuple{KnetArray{Float32,N} where N,KnetArray{Float32,N} where N}}
257
(196608, 256)
16-element Data{Tuple{KnetArray{Float32,N} where N,KnetArray{Float32,N} where N}}
513
(196608, 512)
24-element Data{Tuple{KnetArray{Float32,N} where N,KnetArray{Float32,N} where N}}
769
(196608, 768)
32-element Data{Tuple{KnetArray{Float32,N} where N,KnetArray{Float32,N} where N}}


(K32(256,256,3,32)[0.19607843⋯], K32(32,32,71,32)[0.0⋯])

In [ ]:
#Knet.save("20-12-2020-24kimage-batches.jld2", "24k-image-batches", (dtrn, dval))

In [ ]:
loaded_batches = Knet.load("20-12-2020-24kimage-batches.jld2", "24k-image-batches");
dtrn, dval = loaded_batches
dtrn_x1, dtrn_y1 = first(dtrn)
dval_x1, dval_y1 = first(dval)

In [21]:
# TODO: check resnet implementation for its regularizations
should_use_resnet50 = true
modular_resnet = get_modular_resnet(should_use_resnet50);

In [ ]:
#size(resnet_with_deconv(dtrn_x1))

In [22]:
resnet_with_deconv = Chain(modular_resnet.layers...,  
     Deconv(4, 4, global_num_joints, 2048; padding=1, stride=2, tag="part_detect_deconv"); 
    loss=sigmoid_cross_entropy_loss,
    deeperCutOption=DeeperCutOption(true));

In [23]:
model_end_results(resnet_with_deconv, dtrn_2k, dval_2k)

train_loss: 1.0323536
train_acc: 0.002022879464285714
validation_loss: 1.03175
validation_acc: 0.0011160714285714287


In [ ]:
resnet_with_deconv_conv3_disabled = Chain(modular_resnet.layers...,  
     Deconv(4, 4, 14, 2048; padding=1, stride=2, tag="part_detect_deconv"); 
    loss=sigmoid_cross_entropy_loss,
    deeperCutOption=DeeperCutOption(false));

In [ ]:
model_end_results(resnet_with_deconv_conv3_disabled, dtrn_2k, dval_2k)

In [ ]:
CUDA.reclaim()
CUDA.memory_status() 
GC.gc(true)

In [ ]:
# [(0.005, 1), (0.02, 20), (0.002, 15), (0.001, 15)]
resnetnet_results = trainresults("resnet_with_deconv.jld2", resnet_with_deconv_conv3_disabled, dtrn_2k, dval_2k, [(0.02, 5)]; accuracy_func=modelized_naive_pck_sigm, error_func=nothing);

In [ ]:
resnetnet_results = trainresults("resnet_with_deconv.jld2", resnet_with_deconv, dtrn_2k, dval_2k, [(0.02, 5)]; accuracy_func=modelized_naive_pck_sigm, error_func=nothing);

In [ ]:
#Knet.save("20-+5ep-12-2020-24kimage-resnet-50.jld2", "resnet_with_deconv", resnet_with_deconv)

In [ ]:
#resnet_with_deconv = Knet.load("20-+5ep-12-2020-24kimage-resnet-50.jld2", "resnet_with_deconv");

In [ ]:
modelized_naive_pck_sigm(resnet_with_deconv, dval)

In [ ]:
draw_plots(resnetnet_results)

In [ ]:
sigmoided_output = Array(sigm.(resnet_with_deconv(dtrn_x1)));

In [ ]:
sigmoided_output = Array(sigm.(loadedResNet(dtrn_x1)));

In [ ]:
sigmoided_output[ sigmoided_output .< output_consider_threshold] .= 0  ;

In [ ]:
test_image_order = 7;

In [ ]:
gt_joint_images = draw_score_maps(Array(dtrn_y1), test_image_order);

In [ ]:
pred_joint_images = draw_score_maps(sigmoided_output, test_image_order);

In [ ]:
pred_joint_images[14]

In [ ]:
gt_joint_images[14]

In [ ]:
images = []
for (i, joint_sc) in enumerate(gt_joint_images)
img = show_scmap_on_image(test_image, joint_sc;
        fill_value=1, 
        should_use_scmap_size=true,
    display_name=i)
    push!(images, img)
end

In [ ]:
mosaicify(images; ncol=3)

In [ ]:
show_scmap_on_image(test_image, pred_joint_images[13]; fill_value=1);

In [ ]:
should_use_output_sized = false
test_image = Array(dtrn_x1_2k)[:, :, :, test_image_order];
output_sized_test_image = imresize(test_image, 32, 32);
perm = permutedims( should_use_output_sized ? output_sized_test_image : test_image, [3, 1, 2])
colored = colorview(RGB, perm);
fetch(display(colored));
#colorview(RGB, rand(3,256,256))

In [ ]:
# Experimental Section

In [ ]:
first(dtrn)[1] |> size

In [ ]:
first(dtrn)[2] |> size

In [ ]:
#julia> w0 = KnetArray(randn(Float32, 5, 5, 1, 20));

#julia> x0 = KnetArray(rand(Float32, 24, 24, 20, 100));

#julia> size(deconv4(w0,x0)) # this output does not make it sense, it should give what I want
#(28, 28, 20, 100)
#

w0 = KnetArray(randn(Float32, 5, 5, 1, 20));
 x0 = KnetArray(rand(Float32, 4, 4, 2048, 32));

 # this output does not make it sense, it should give what I want
#(28, 28, 20, 100)

In [ ]:
dec_layer = Deconv(4, 4, 14, 2048; padding=1, stride=2, atype=Knet.atype())

In [ ]:
mock = randn(Float32, 16,16, 2048, 32);
mock = convert(Knet.atype(), mock);

In [ ]:
size(dec_layer(  mock ))

In [ ]:
size(deconv4(w0,x0))

In [ ]:
tc = Conv2(3, 3, 3, 3)

In [ ]:
fb = first(dtrn)[1];

In [ ]:
res_tc = tc2(fb) |> tc;

In [ ]:
mid = res_tc |> tc2

In [ ]:
w1 = KnetArray(randn(Float32, 3,3, 1, 3));

In [ ]:
#deconv4(w1, mid)
size(deconv4(w1, mid; padding=1))

In [ ]:
tc2 = ResLayerConv(3, 3, 3, 3; stride=2, padding=1)

In [ ]:
res_tc2 = res_tc |> tc2;

In [ ]:
size(res_tc)

In [ ]:
tc2(fb) |> tc2 |> tc2;